In [30]:
import os
from os.path import join as oj
import sys, time
sys.path.insert(1, oj(sys.path[0], '..'))  # insert parent path
import seaborn as sns
from sklearn.model_selection import train_test_split
from regression_dsets_large_names import regression_dsets_large_names
import numpy as np
import matplotlib.pyplot as plt
import pmlb
from tqdm import tqdm
from copy import deepcopy
import pickle as pkl
import pandas as pd
import data
import fit
from process_results import process_results, aggregate_results, best_ridge

%matplotlib inline
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
!python process_results.py

In [31]:
save_dir = 'results'
os.makedirs(save_dir, exist_ok=True)
parent_dir = '/scratch/users/vision/yu_dl/raaz.rsk/double_descent/df' # basic, pmlb_0, clustered, pmlb_1
folders = os.listdir(parent_dir)
print(folders)

['pmlb_1', 'decay', 'pmlb_0', 'decay2', 'spike', 'clustered', 'basic']


# linear plots

In [32]:
# for folder in folders:
folder = 'decay2'
out_dir = oj(parent_dir, folder)
df = pd.read_pickle(oj(out_dir, 'processed.pkl'))


In [33]:
df.keys()

Index(['ratio', 'bias', 'var', 'wnorm', 'mse_train', 'mse_test', 'num_nonzero',
       'mse_noiseless', 'df1', 'df2', 'df3', 'mse_zero'],
      dtype='object')

In [38]:
df.index

MultiIndex(levels=[['gaussian'], [0], ['gaussian'], ['ols', 'ridge'], [0.0, 0.01, 0.1, 1.0, 10.0], [1], [0.1], ['gaussian'], ['decay'], [0.0, 1.0, 2.0, 4.0]],
           codes=[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [0, 0, 0, 0, 1, 1, 1, 1, 2, 2, 2, 2, 3, 3, 3, 3, 4, 4, 4, 4], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3]],
           names=['dset', 'dset_num', 'beta_type', 'model_type', 'reg_param', 'beta_norm', 'noise_mult', 'noise_distr', 'iid', 'cov_param'])

Float64Index([0.0, 1.0, 2.0, 4.0], dtype='float64', name='cov_param')

In [ ]:
row_best_ridge = best_ridge(df) # contains df1, df2, df3
len_ridge = row_best_ridge.df1.size

for param_num in [0, 1, 2, 4]:

    R, C = 1, 2
    SIZE = 5
    plt.figure(figsize=(C * SIZE, R * SIZE))
    plt.subplot(R, C, 1)
    plt.axhline(df.mse_zero.values[0], lw=4, color='gray', alpha=0.4, label='trivial')

    for name, curve in tqdm(df.iterrows()):
        ind = df.index.names.index('cov_param')
        cov_param = name[ind]

        if cov_param == param_num:
        #     print(name, curve)
            model_type = name[3]
            reg_param = name[4]
            l = str(model_type) + ' ' + str(reg_param) + ' ' + str(name[-1])
            if model_type == 'ols':
                ls = '--'
            elif model_type == 'ridge':
                ls = '-'
            elif model_type == 'linear_sta':
                ls = '-.'
            elif model_type == 'lasso':
                ls = ':'

            if not model_type == 'linear_sta':


                kwargs = {'label': l, 'alpha': 0.5, 'lw': 2, 'linestyle': ls}

                plt.subplot(R, C, 1)
            #     plt.plot(curve.ratio, curve.mse_test, 
                plt.plot(curve.ratio, curve.mse_noiseless, **kwargs) #np.square(curve.bias) + curve['var'], **kwargs)
                plt.ylabel('test mse')
                plt.xlabel('p / n')

                plt.subplot(R, C, 2)
                plt.plot(curve.ratio, curve.mse_train, **kwargs)
                plt.ylabel('train mse')
                plt.xlabel('p / n')

                '''
                plt.subplot(R, C, 4)
                plt.plot(row_best_ridge.df1, curve.mse_noiseless[:len_ridge], **kwargs)
                plt.ylabel('test mse')
                plt.xlabel('df1: $tr(S S^T)$')

                plt.subplot(R, C, 5)
                plt.plot(row_best_ridge.df2, curve.mse_noiseless[:len_ridge], **kwargs)
                plt.ylabel('test mse')
                plt.xlabel('df2: $tr(2 * S - S^T S)$')

                plt.subplot(R, C, 6)
                plt.plot(row_best_ridge.df3, curve.mse_noiseless[:len_ridge], **kwargs)
                plt.ylabel('test mse')
                plt.xlabel('df3: $tr(S)$')
                '''


                '''
                plt.subplot(R, C, 3)
                plt.plot(curve.ratio, np.square(curve.bias), **kwargs)
                plt.ylabel('bias')

                plt.subplot(R, C, 4)
                plt.plot(curve.ratio, curve['var'], **kwargs)
                plt.ylabel('var')
                '''

                '''
                # wnorm also recorded
                plt.subplot(R, C, 3)
                plt.plot(curve.ratio, curve['wnorm'], **kwargs)
                plt.ylabel('wnorm')
                plt.xlabel('p / n')

                plt.subplot(R, C, 4)
                plt.plot(curve.ratio, curve['num_nonzero'], **kwargs)
                plt.ylabel('num_nonzero')
                plt.xlabel('p / n')
                '''

    for i in range(1, R * C + 1):
        plt.subplot(R, C, i)
        plt.xscale('log')
        plt.yscale('log')

    #     s = f'{model_type}_{dset}_{dset_name}+p={num_features}+lambda={reg_param}+noise_mult={noise_mult}'
    plt.subplot(R, C, 1)
    plt.legend()

    '''
    plt.subplot(R, C, 3)
    ratios = row_best_ridge.ratio
    n = 500 / ratios
    p = 0 * ratios + 500
    plt.plot(ratios, np.minimum(n, p), label='min(n, p)', alpha=0.5)
    plt.plot(ratios, row_best_ridge.df1, label='df1')
    plt.plot(ratios, row_best_ridge.df2, label='df2')
    plt.plot(ratios, row_best_ridge.df3, label='df3')
    plt.xlabel('p / n')
    plt.legend()
    '''


    s = folder
    # plt.suptitle(s)
    plt.tight_layout()
    # plt.show()
    plt.savefig(oj(save_dir, s + f'_{param_num}.pdf'))
    plt.show()
    plt.close()